In [13]:
import requests
import json
import time
import pandas as pd
from tmdbv3api import TMDb, TV
from tmdbv3api.exceptions import TMDbException

In [ ]:
def get_show_details(api_key, query):
    # Base URL for the TMDb API
    base_url = "https://api.themoviedb.org/3/search/tv"

    # Parameters for the API request
    params = {
        'api_key': api_key,
        'query': query
    }

    # Make the API request
    response = requests.get(base_url, params=params)

    # If the request was successful, parse the JSON
    if response.status_code == 200:
        return response.json()
    else:
        return None

def main():
    # Replace with your TMDb API key
    api_key = 'f082066c7a8bedc4aab8ca888b9cc562'

    # TV show to search for
    query = "Grey's Anatomy"

    # Get the show details
    show_details = get_show_details(api_key, query)

    if show_details:
        print(json.dumps(show_details, indent=4))
    else:
        print("API request failed.")

if __name__ == "__main__":
    main()

{
    "page": 1,
    "results": [
        {
            "adult": false,
            "backdrop_path": "/ds1n5P0Y92cTh6UpbcA89hxM96a.jpg",
            "genre_ids": [
                18
            ],
            "id": 1416,
            "origin_country": [
                "US"
            ],
            "original_language": "en",
            "original_name": "Grey's Anatomy",
            "overview": "Follows the personal and professional lives of a group of doctors at Seattle\u2019s Grey Sloan Memorial Hospital.",
            "popularity": 736.279,
            "poster_path": "/daSFbrt8QCXV2hSwB0hqYjbj681.jpg",
            "first_air_date": "2005-03-27",
            "name": "Grey's Anatomy",
            "vote_average": 8.252,
            "vote_count": 9540
        },
        {
            "adult": false,
            "backdrop_path": "/adxil95KlgPo2xhwJQayGVB43C6.jpg",
            "genre_ids": [
                18
            ],
            "id": 79897,
            "origin_country": [
     

In [15]:
# Function to get the ID of the TV show
def get_show_id(api_key, query):
    url = f"https://api.themoviedb.org/3/search/tv?api_key={api_key}&query={query}"
    response = requests.get(url)
    data = response.json()
    if 'results' in data and data['results']:
        # Assuming the first show in the results is the one we're looking for
        return data['results'][0]['id']
    return None  # Handle no results or errors appropriately

# Function to get all seasons of the TV show
def get_seasons(api_key, show_id):
    url = f"https://api.themoviedb.org/3/tv/{show_id}?api_key={api_key}"
    response = requests.get(url)
    data = response.json()
    if 'seasons' in data:
        return data['seasons']
    return None  # Handle no results or errors appropriately

# Function to get all episodes of a season
def get_episodes(api_key, show_id, season_number):
    url = f"https://api.themoviedb.org/3/tv/{show_id}/season/{season_number}?api_key={api_key}"
    response = requests.get(url)
    data = response.json()
    if 'episodes' in data:
        return data['episodes']
    return None  # Handle no results or errors appropriately


def get_episode_details(episode):
    # Extract the required details from the episode data
    details = {
        'episode_name': episode['name'],
        'air_date': episode['air_date'],
        'overview': episode['overview'],
        'runtime': episode.get('runtime', 'N/A'),  # Runtime might not be available for all episodes
        'guest_stars': ', '.join([star['name'] for star in episode.get('guest_stars', [])]),
        'crew': ', '.join([f"{member['name']} ({member['job']})" for member in episode.get('crew', [])]),
        'cast': ', '.join([f"{member['name']} as {member['character']}" for member in episode.get('cast', [])]),
        'image': episode.get('still_path', 'N/A')  # Handle case where image is not available
    }
    guest_stars = []
    if 'guest_stars' in episode and episode['guest_stars']:
        for star in episode['guest_stars']:
            guest_stars.append({
                'Star Name': star['name'],
                'Popularity': star['popularity']
            })
    return details,guest_stars


In [ ]:
def main():
    api_key = 'f082066c7a8bedc4aab8ca888b9cc562'  # replace with your TMDb API key
    show_name = "Grey's Anatomy"

    # Step 1: Get the TV show ID
    show_id = get_show_id(api_key, show_name)
    if not show_id:
        print(f"Could not find a show with the name {show_name}")
        return

    # Step 2: Get all the seasons for the TV show
    seasons = get_seasons(api_key, show_id)
    if not seasons:
        print(f"Could not find seasons for the show {show_name}")
        return

    columns = ['season', 'episode_number', 'episode_name', 'air_date', 'overview', 'runtime', 'guest_stars', 'crew', 'cast', 'image']
    episode_df = pd.DataFrame(columns=columns)
    columns1 = ['season', 'episode_number', 'Episode Name','Star Name','Popularity']
    guests_df = pd.DataFrame(columns=columns1)

    # Step 3: For each season, get all episodes
    for season in seasons:
        season_number = season['season_number']
        episodes = get_episodes(api_key, show_id, season_number)
        if not episodes:
            print(f"Could not find episodes for season {season_number} of the show {show_name}")
            continue

        # Store episode details in the DataFrame
        for episode in episodes:
            details,guest_details = get_episode_details(episode)
            details['season'] = season_number
            details['episode_number'] = episode['episode_number']
            episode_df = episode_df.append(details, ignore_index=True)
            for guest in guest_details:
              guest['season'] = season_number
              guest['episode_number'] = episode['episode_number']
              guest['Episode Name'] = details['episode_name']
              guests_df = guests_df.append(guest, ignore_index=True)

            # Handle rate limits and be respectful to the API server
            time.sleep(0.5)

    # Save the DataFrame to a CSV file
    episode_df.to_csv('greys_anatomy_episodes.csv', index=False)
    guests_df.to_csv('greys_anatomy_actors.csv', index=False)

if __name__ == "__main__":
    main()

<ipython-input-11-86c320582052>:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  episode_df = episode_df.append(details, ignore_index=True)
<ipython-input-11-86c320582052>:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  episode_df = episode_df.append(details, ignore_index=True)
<ipython-input-11-86c320582052>:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  episode_df = episode_df.append(details, ignore_index=True)
<ipython-input-11-86c320582052>:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  episode_df = episode_df.append(details, ignore_index=True)
<ipython-input-11-86c320582052>:88: FutureWarning: The frame.append method is de

In [ ]:
show_name = "Grey's Anatomy"
show_id = get_show_id('f082066c7a8bedc4aab8ca888b9cc562',show_name)
print(show_id)
episodes = get_episodes('f082066c7a8bedc4aab8ca888b9cc562', show_id, 1)
print(episodes)

1416
[{'air_date': '2005-03-27', 'episode_number': 1, 'episode_type': 'standard', 'id': 64383, 'name': "A Hard Day's Night", 'overview': "Meet Meredith Grey. Daughter of famous surgeon Ellis Grey and currently recovering from a one-night stand. She shoos away her evening's companion, saying she'll be late for her first day of work. She is a first year intern at Seattle Grace Hospital and is working alongside some colorful characters. Cristina Yang is a highly competitive Stanford graduate who befriends Meredith. George O'Malley is a quirky warm hearted man who learns a lesson or two on his first day. Izzie Stevens is a former model who doesn't get any special treatment on her first day. And Alex Karev is an arrogant jerk who misdiagnoses a patient on his first shift. Overseeing them is Miranda Bailey, (not-so-) affectionately dubbed 'The Nazi'. Meredith also gets a surprise when she realizes that her friend from the night before is a co-worker, and even worse, her boss.", 'production_c

In [ ]:
for episode in episodes:
  print(episode)
  break

{'air_date': '2005-03-27', 'episode_number': 1, 'episode_type': 'standard', 'id': 64383, 'name': "A Hard Day's Night", 'overview': "Meet Meredith Grey. Daughter of famous surgeon Ellis Grey and currently recovering from a one-night stand. She shoos away her evening's companion, saying she'll be late for her first day of work. She is a first year intern at Seattle Grace Hospital and is working alongside some colorful characters. Cristina Yang is a highly competitive Stanford graduate who befriends Meredith. George O'Malley is a quirky warm hearted man who learns a lesson or two on his first day. Izzie Stevens is a former model who doesn't get any special treatment on her first day. And Alex Karev is an arrogant jerk who misdiagnoses a patient on his first shift. Overseeing them is Miranda Bailey, (not-so-) affectionately dubbed 'The Nazi'. Meredith also gets a surprise when she realizes that her friend from the night before is a co-worker, and even worse, her boss.", 'production_code': 

In [6]:
episodes_df = pd.read_csv('greys_anatomy_episodes.csv')
guests_df=pd.read_csv('greys_anatomy_actors.csv')

In [7]:
episodes_df

,season,episode_number,episode_name,air_date,overview,runtime,guest_stars,crew,cast,image
0,0,1,Straight To The Heart,2006-01-08,Recap of series to date.,43,NaN,NaN,NaN,/t8fRHKy5jGUq7U1EE8XUpN7DJyQ.jpg
1,0,2,Under Pressure,NaN,"This is a recap episode, otherwise known as a ...",43,NaN,NaN,NaN,/bTmIUauHG6gPRHqqPLpdIa05Jqv.jpg
2,0,3,Complications of the Heart,NaN,Another recap episode narrated by Joe the Bart...,43,NaN,NaN,NaN,/kxRVPAIVyPsfN8s7nPCqZdCOco9.jpg
3,0,4,Every Moment Counts,NaN,Quick summary of all the important details fro...,43,NaN,"Chris Van Dusen (Writer), Tamra Davis (Directo...",NaN,/ocwrUUusLWE8WejNIxSXSye00WC.jpg
4,0,5,Come Rain or Shine: From Grey's Anatomy to Pri...,NaN,"""Come Rain or Shine: From Grey's Anatomy to Pr...",43,NaN,Marti Noxon (Consulting Producer),NaN,/6EUseSD5cZfKIzAo6OhUcFhM1H8.jpg
...,...,...,...,...,...,...,...,...,...,...
425,19,16,Gunpowder and Lead,2023-04-20,Amelia takes her personal problems out on her ...,43,"Jason George, Stefania Spampinato, Jaicy Ellio...","Michelle Lirtzman (Writer), Morenike Joela Eva...",NaN,/gBICdDEBg9z5bv6BYJXvmuXwNux.jpg
426,19,17,Come Fly with Me,2023-05-04,Teddy calls an emergency meeting to discuss th...,43,"Jaicy Elliot, Juliet Mills, Jasmine McLeish, S...","Amyn Kaderali (Director), Kingsley Ume (Writer)",NaN,/cT30I8qYp24TzZq31XXPgDv4e7j.jpg
427,19,18,Ready to Run,2023-05-11,Richard and Teddy make an exciting announcemen...,44,"Jaicy Elliot, Juliet Mills, Olivia Negron, Sam...","Julie Wong (Writer), Allison Liddi-Brown (Dire...",NaN,/bfgko2PtURa9mkKlWQCaIywK50S.jpg
428,19,19,Wedding Bell Blues,2023-05-18,Simone's wedding day arrives as Jo and Link's ...,42,"Jaicy Elliot, Debbie Allen, Marla Gibbs, Julie...","Krista Vernoff (Writer), Meg Marinis (Writer),...",NaN,/s2Pn5OjSCRuCno1nTa5YabvhS2c.jpg


In [8]:
guests_df

,season,episode_number,Episode Name,Star Name,Popularity
0,1,1,A Hard Day's Night,Sendhil Ramamurthy,5.369
1,1,1,A Hard Day's Night,Skyler Shaye,2.141
2,1,1,A Hard Day's Night,Laura Carson,0.621
3,1,1,A Hard Day's Night,David Vegh,2.139
4,1,1,A Hard Day's Night,Randall Arney,1.847
...,...,...,...,...,...
1917,19,20,Happily Ever After?,Sam Page,11.572
1918,19,20,Happily Ever After?,Kwame Patterson,1.921
1919,19,20,Happily Ever After?,William Martinez,1.336
1920,19,20,Happily Ever After?,Dorian Gregory,4.422


In [30]:
from google.colab import files

files.download('greys_anatomy_episodes.csv')
files.download('greys_anatomy_actors.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [1]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))


Saving greys_anatomy_actors.csv to greys_anatomy_actors.csv
Saving greys_anatomy_episodes.csv to greys_anatomy_episodes.csv
User uploaded file "greys_anatomy_actors.csv" with length 87808 bytes
User uploaded file "greys_anatomy_episodes.csv" with length 215619 bytes


In [2]:
def analyze_missing_values(df):
    # Check for missing values in the dataframe
    missing_values = df.isnull().sum()

    # Calculate the percentage of missing values
    missing_percentage = (df.isnull().sum() / len(df)) * 100

    # Create a dataframe from the missing_values and missing_percentage
    missing_data = pd.DataFrame({'Total Missing Values': missing_values, '% of Missing Values': missing_percentage})

    # Sort the dataframe based on the highest percentage of missing values
    missing_data_sorted = missing_data[missing_data.iloc[:, 1] != 0].sort_values(by='% of Missing Values', ascending=False).round(1)

    # Return the resulting dataframe
    return missing_data_sorted


In [9]:
missing_data_analysis_ep = analyze_missing_values(episodes_df)
print(missing_data_analysis_ep)
missing_data_analysis_gu = analyze_missing_values(guests_df)
print(missing_data_analysis_gu)

             Total Missing Values  % of Missing Values
cast                          430                100.0
guest_stars                    96                 22.3
image                           6                  1.4
air_date                        4                  0.9
crew                            4                  0.9
Empty DataFrame
Columns: [Total Missing Values, % of Missing Values]
Index: []


In [10]:
episodes_df.drop(columns=['cast','guest_stars'])

,season,episode_number,episode_name,air_date,overview,runtime,crew,image
0,0,1,Straight To The Heart,2006-01-08,Recap of series to date.,43,NaN,/t8fRHKy5jGUq7U1EE8XUpN7DJyQ.jpg
1,0,2,Under Pressure,NaN,"This is a recap episode, otherwise known as a ...",43,NaN,/bTmIUauHG6gPRHqqPLpdIa05Jqv.jpg
2,0,3,Complications of the Heart,NaN,Another recap episode narrated by Joe the Bart...,43,NaN,/kxRVPAIVyPsfN8s7nPCqZdCOco9.jpg
3,0,4,Every Moment Counts,NaN,Quick summary of all the important details fro...,43,"Chris Van Dusen (Writer), Tamra Davis (Directo...",/ocwrUUusLWE8WejNIxSXSye00WC.jpg
4,0,5,Come Rain or Shine: From Grey's Anatomy to Pri...,NaN,"""Come Rain or Shine: From Grey's Anatomy to Pr...",43,Marti Noxon (Consulting Producer),/6EUseSD5cZfKIzAo6OhUcFhM1H8.jpg
...,...,...,...,...,...,...,...,...
425,19,16,Gunpowder and Lead,2023-04-20,Amelia takes her personal problems out on her ...,43,"Michelle Lirtzman (Writer), Morenike Joela Eva...",/gBICdDEBg9z5bv6BYJXvmuXwNux.jpg
426,19,17,Come Fly with Me,2023-05-04,Teddy calls an emergency meeting to discuss th...,43,"Amyn Kaderali (Director), Kingsley Ume (Writer)",/cT30I8qYp24TzZq31XXPgDv4e7j.jpg
427,19,18,Ready to Run,2023-05-11,Richard and Teddy make an exciting announcemen...,44,"Julie Wong (Writer), Allison Liddi-Brown (Dire...",/bfgko2PtURa9mkKlWQCaIywK50S.jpg
428,19,19,Wedding Bell Blues,2023-05-18,Simone's wedding day arrives as Jo and Link's ...,42,"Krista Vernoff (Writer), Meg Marinis (Writer),...",/s2Pn5OjSCRuCno1nTa5YabvhS2c.jpg


In [11]:
episodes_df['crew'][6]

'Sarah Drew (Director), Barbara Kaye Friend (Writer)'

In [16]:
tv_id = get_show_id('f082066c7a8bedc4aab8ca888b9cc562', "Grey's Anatomy")

In [23]:
tv_id

1416

In [25]:
def get_episode_rating(api_key, series_id, season_number, episode_number):
    url = f"https://api.themoviedb.org/3/tv/{series_id}/season/{season_number}/episode/{episode_number}"
    params = {
        "api_key": api_key,
        "language": "en-US"  # Optional: Change this to the language of your choice
    }
    response = requests.get(url, params=params)
    data = response.json()
    return data.get("vote_average")  # This retrieves the rating of the episode


In [26]:
episodes_df['rating'] = None
api_key = "f082066c7a8bedc4aab8ca888b9cc562"
series_id = "1416"
# Get the rating for each episode and add it to the DataFrame
for index, row in episodes_df.iterrows():
    rating = get_episode_rating(api_key, tv_id, row['season'], row['episode_number'])
    episodes_df.at[index, 'rating'] = rating


In [27]:
episodes_df

,season,episode_number,episode_name,air_date,overview,runtime,guest_stars,crew,cast,image,rating
0,0,1,Straight To The Heart,2006-01-08,Recap of series to date.,43,NaN,NaN,NaN,/t8fRHKy5jGUq7U1EE8XUpN7DJyQ.jpg,6.7
1,0,2,Under Pressure,NaN,"This is a recap episode, otherwise known as a ...",43,NaN,NaN,NaN,/bTmIUauHG6gPRHqqPLpdIa05Jqv.jpg,7.7
2,0,3,Complications of the Heart,NaN,Another recap episode narrated by Joe the Bart...,43,NaN,NaN,NaN,/kxRVPAIVyPsfN8s7nPCqZdCOco9.jpg,6.0
3,0,4,Every Moment Counts,NaN,Quick summary of all the important details fro...,43,NaN,"Chris Van Dusen (Writer), Tamra Davis (Directo...",NaN,/ocwrUUusLWE8WejNIxSXSye00WC.jpg,7.0
4,0,5,Come Rain or Shine: From Grey's Anatomy to Pri...,NaN,"""Come Rain or Shine: From Grey's Anatomy to Pr...",43,NaN,Marti Noxon (Consulting Producer),NaN,/6EUseSD5cZfKIzAo6OhUcFhM1H8.jpg,7.0
...,...,...,...,...,...,...,...,...,...,...,...
425,19,16,Gunpowder and Lead,2023-04-20,Amelia takes her personal problems out on her ...,43,"Jason George, Stefania Spampinato, Jaicy Ellio...","Michelle Lirtzman (Writer), Morenike Joela Eva...",NaN,/gBICdDEBg9z5bv6BYJXvmuXwNux.jpg,7.0
426,19,17,Come Fly with Me,2023-05-04,Teddy calls an emergency meeting to discuss th...,43,"Jaicy Elliot, Juliet Mills, Jasmine McLeish, S...","Amyn Kaderali (Director), Kingsley Ume (Writer)",NaN,/cT30I8qYp24TzZq31XXPgDv4e7j.jpg,7.0
427,19,18,Ready to Run,2023-05-11,Richard and Teddy make an exciting announcemen...,44,"Jaicy Elliot, Juliet Mills, Olivia Negron, Sam...","Julie Wong (Writer), Allison Liddi-Brown (Dire...",NaN,/bfgko2PtURa9mkKlWQCaIywK50S.jpg,0.0
428,19,19,Wedding Bell Blues,2023-05-18,Simone's wedding day arrives as Jo and Link's ...,42,"Jaicy Elliot, Debbie Allen, Marla Gibbs, Julie...","Krista Vernoff (Writer), Meg Marinis (Writer),...",NaN,/s2Pn5OjSCRuCno1nTa5YabvhS2c.jpg,8.0


In [29]:
 episodes_df.to_csv('greys_anatomy_episodes_withRating.csv', index=False)

In [31]:
from google.colab import files

files.download('greys_anatomy_episodes_withRating.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>